In [1]:
import requests
from lxml import etree
import os

In [9]:
with open('emt-data/import_transkribus_letters.csv', "r") as inp:
    letters = inp.readlines()

In [10]:
letters = [x.strip() for x in letters]

In [11]:
letters_import = []
for x in letters:
    if x not in letters_import:
        letters_import.append(x)

In [2]:
transkribus_base_url = "https://transkribus.eu/TrpServer/rest/"

In [3]:
def login_transkribus(user, password):
    res = requests.post(f"{transkribus_base_url}auth/login", data={"user": user, "pw": password})
    if res.status_code == 200:
        tree = etree.fromstring(res.content)
        sessionid = tree.xpath('/trpUserLogin/sessionId/text()')
        cookies = dict(JSESSIONID=sessionid[0])
        return cookies
    else:
        return False


In [22]:
def search_for_document(title, collectionid, cookies):
    res = requests.get(f"{transkribus_base_url}collections/findDocuments", cookies=cookies, params={"collId": collectionid, "title": title, })
    return res.json()

In [25]:
def search_for_collection(title, cookies):
    res = requests.get(f"{transkribus_base_url}collections/listByName", cookies=cookies, params={"name": title}, headers={"Accept": "application/json"})
    return res.json()

In [23]:
def create_transkribus_collection(name, cookies):
    res = requests.post(f"{transkribus_base_url}collections/createCollection", cookies=cookies, params={'collName': name})
    if res.status_code == 200:
        return res.content
    else:
        print("error: ", res.status_code, res.content)
        return False

In [24]:
def transkribus_upload_mets_file(mets_url, collectionid, cookies):
    res = requests.post(f"{transkribus_base_url}collections/{collectionid}/createDocFromMetsUrl", cookies=cookies, params={'fileName': mets_url})
    if res.status_code == 200:
        return True
    else:
        print("Error: ", res.status_code, res.content)
        return False


In [16]:
import_fin = dict()
for let in letters_import:
    kst = "".join(let.split("_")[:-1])
    if kst not in import_fin.keys():
        import_fin[kst] = [let]
    else:
        import_fin[kst].append(let)

In [28]:
import_fin

{'Kastenblau442b': ['Kasten_blau_44_2b_0002',
  'Kasten_blau_44_2b_0005',
  'Kasten_blau_44_2b_0009',
  'Kasten_blau_44_2b_0011',
  'Kasten_blau_44_2b_0012'],
 'Kastenblau442d': ['Kasten_blau_44_2d_0002',
  'Kasten_blau_44_2d_0004',
  'Kasten_blau_44_2d_0007',
  'Kasten_blau_44_2d_0009',
  'Kasten_blau_44_2d_0011',
  'Kasten_blau_44_2d_0012',
  'Kasten_blau_44_2d_0013',
  'Kasten_blau_44_2d_0015',
  'Kasten_blau_44_2d_0017',
  'Kasten_blau_44_2d_0020',
  'Kasten_blau_44_2d_0022',
  'Kasten_blau_44_2d_0025',
  'Kasten_blau_44_2d_0026',
  'Kasten_blau_44_2d_0027',
  'Kasten_blau_44_2d_0029',
  'Kasten_blau_44_2d_0031',
  'Kasten_blau_44_2d_0034',
  'Kasten_blau_44_2d_0038',
  'Kasten_blau_44_2d_0039',
  'Kasten_blau_44_2d_0041',
  'Kasten_blau_44_2d_0042',
  'Kasten_blau_44_2d_0043',
  'Kasten_blau_44_2d_0045',
  'Kasten_blau_44_2d_0047'],
 'Kastenblau443d': ['Kasten_blau_44_3d_0002',
  'Kasten_blau_44_3d_0012',
  'Kasten_blau_44_3d_0015',
  'Kasten_blau_44_3d_0019'],
 'Kastenblau443f': 

In [8]:
cookie = login_transkribus(os.environ.get('TRANSKRIBUS_USER'), os.environ.get('TRANSKRIBUS_PASSWORD'))

In [27]:
for kasten in import_fin.keys():
    search_col = search_for_collection(kasten, cookie)
    if len(search_col) > 0:
        col = search_col[0]["collId"]
    else:
        col = create_transkribus_collection(kasten, cookie)
        col = col.decode("utf8")
        if not col:
            break
    print("col")
    for let in import_fin[kasten]:
        search = search_for_document(let, col.decode{"utf8"}, cookie)
        if len(search) == 0:
            let2 = transkribus_upload_mets_file(f"https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id={let}", col, cookie)

col
col
col
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3d_0019'
col
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0032'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0033'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0037'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0040'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0042'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0045'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0046'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0048'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_blau_44_3f_0050'
Error:  400 b'https://goobi.acdh.oeaw.ac.at/viewer/metsresolver?id=Kasten_